In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
imdb_sentences = []
imdb_labels = []

In [ ]:
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split='train'))

: 

In [ ]:
for item in train_data:
	imdb_sentences.append(str(item['text']))
	imdb_labels.append(item['label'])

len(imdb_sentences), len(imdb_labels)



In [ ]:
training_sentences = imdb_sentences[:20000]
test_sentences = imdb_sentences[20000:]

training_labels = imdb_labels[:20000]
test_labels = imdb_labels[20000:]

len(training_sentences), len(test_sentences), len(training_labels), len(test_labels)

# Tokenize

In [ ]:
# Tokenize the sentences
phrase = "Je suis au marché"
tokens = phrase.split(' ')
# tout les mots sont séparés par un espace
# tout en miniscule 


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(training_sentences) 
#tokenizer.word_index
#tokenizer.index_word

In [ ]:
tokenizer.texts_to_sequences(training_sentences)
# le probleme est que des mots dans le test set n'existe pas dans le training set
# notion de OOV (Out Of Vocabulary)
tokenizer2 = Tokenizer(num_words=100, oov_token='<OOV>')
tokenizer2.fit_on_texts(training_sentences)


In [ ]:
# Padding et Truncating du texte
phrases = [ ]
tokenizer = Tokenizer(num_words = 100,oov_token='<OOV>')
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)

sequences

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


# default tout les zeros a gauches (les phrases font la meme taille), prepadding
# padding='post' pour postpadding
# maxlen=5 pour limiter la taille des phrases
# truncating='post' pour tronquer les phrases a la fin
padded = pad_sequences(sequences, padding='post', truncating='post', maxlen=5)

In [ ]:
# les stopwords sont des mots qui n'ont pas de sens (the, a, an, in, on, ...) = dead words (mots vides)
# je vais au marche le Jeudi => je vais marche Jeudi

stopwords = ['au', "le"]
words = phrase.split(' ')
phrase_cleaned = ""

for word in words:
	if word not in stopwords:
		phrase_cleaned.append(word)
print(''.join(phrases_cleaned))

In [ ]:
# utiliser package NLTK pour les stopwords
# on a un tableau numpy
# calcul du sentiment
# chaque mot a un poids (positif ou negatif), on fait la somme des poids
# si positif => sentiment positif
# si negatif => sentiment negatif
# difficile pour evaluer le sarcasme (+1 et -1 = Neutre)
# autre strategie : plusieurs poids pour chaque mot [categories, sentiment, nombre de lettres, occurance]



# Les Embeddings - Intuition
vocab_size, em

plusieurs poids par mot
classification binaire compare au label

example :
Royaute + Homme = Roi
Roi - Homme + Femme = Reine

In [ ]:
vocab_size, embedding_dim = 10000, 16


In [ ]:
#Training 
training_sentences[42] 	#un commentaire
training_labels[42] 	#label du commentaire
#nettoyage du texte
tokenizer = Tokenizer(num_words = 20000, oov_token='<OOV>')
tokenizer.fit_on_texts(training_sentences)
#len(tokenizer.word_index) #nombre de mots dans le dictionnaire = too much reduire a 20000
training_sentences = tokenizer.texts_to_sequences(training_sentences)
#on choisi 15 mots par phrase 
training_padded = pad_sequences(training_sentences, maxlen=15, padding='post', truncating='post')

test_sentences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(tokenizer.texts_to_sequences(test_sentences), maxlen=15, padding='post', truncating='post')

In [ ]:
#numpy array
import numpy as np 

training_labels = np.array(training_labels)
test_labels = np.array(test_labels)

In [ ]:
model = tf.keras.models.Sequentials(
	[
		tf.keras.layers.Embedding(20000, 20),
		tf.keras.layers.GlobalAveragePooling1D(),
		tf.keras.layers.Dense(8,activation='relu'),
		tf.keras.layers.Dense(1, activation='sigmoid')
	])

# model.summary() donne le total de parametres

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_ckp = tf.keras.callbacks.ModelCheckpoint(filepath = 'model.h5',
									monitor="val_accuracy",
									mode="max",
									save_best_only=True)
stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
										patience=3,
										restore_best_weights=True)

In [ ]:
h = model.fit(training_padded, training_labels,
			  epochs=50,
			  validation_data=(test_padded, test_labels),
			  callbacks=[model_ckp, stop])
# ajouter un callback pour arreter l'entrainement si la validation loss ne diminue pas
# ajouter un callback pour sauvegarder le meilleur model


In [ ]:
#plot le graph de l'accuracy et de la loss
import matplotlib.pyplot as plt

def plot_graphs(history, string):
	plt.plot(history.history[string])
	plt.plot(history.history['val_'+string])
	plt.xlabel('Epochs')
	plt.ylabel(string)
	plt.legend([string, 'val_'+string])
	plt.show()

plot_graphs(h, 'accuracy')
plot_graphs(h, 'loss')
# on peut voir que le model commence a overfitting 
# training progress but validation progress is not good

# Vocab size

In [ ]:
#Astuces pour ammeliorer le model
#1. Augmenter le nombre de mots dans le dictionnaire
# si vocab_size est trop grand, pas assez d'entrainements (mot inutiles)
# check la frequence des mots
wc = tokenizer.word_counts
wc = sorted(wc.items(), key=lambda x:x[1], reverse=True) 
# prends les mots les plus frequents 
import pandas as pd
pd.Datarame(wc, columns=['mots', 'frequence'])
df[df['frequence'] > 10]


In [ ]:
test_tok = Tokenizer()
test_tok.fit_on_texts(test_sentences)
test_words = test_tok.word_index.keys()
train_words = df['mots'].tolist()
len(train_words), len(test_words)

inter = set(train_words).intersection(set(test_words)) #tout les mots en commun
len(inter) # 33'000 mots en commun
vocab_size = 29000

# Dim Embeddings

In [ ]:
# utiliser racine 4eme du vocab_size pour la dimension de l'embedding
embedding_dim = np.power(vocab_size, 1/4) # ~13


## Change Architecture

In [ ]:
model = tf.keras.models.Sequentials(
	[
		tf.keras.layers.Embedding(vocab_size, embedding_dim),
		tf.keras.layers.GlobalAveragePooling1D(),
		tf.keras.layers.Dense(5,activation='relu'),
		tf.keras.layers.Dense(3, activation='relu'),
		tf.keras.layers.Dense(1, activation='sigmoid')
	])

# plus longtemps
# plus de mots
# optimizer


## change taille maximal phrase


In [ ]:
# on coupe les phrases en max 15 mots
tailles = []
for sent in training_sentences:
	tailles.append(len(sent.split(" ")))
np.array(tailles).mean() # 233 mots par phrase
np.array(tailles).max() # 2470 mots par phrase
np.array(tailles).min() # 4 mots par phrase
np.array(tailles).median() # 174 mots par phrase
# adapter maxlen = 100 au lieu de 15
# rajouter aussi du dropout (randomly drop some neurons to prevent overfitting)
model = tf.keras.models.Sequentials(
	[
		embded
		tf.keras.layers.Embedding(vocab_size, embedding_dim),
		tf.keras.layers.GlobalAveragePooling1D(),
		tf.keras.layers.Dense(5,activation='relu'),
		tf.keras.layers.Dropout(0.25),
		tf.keras.layers.Dense(3, activation='relu'),
		tf.keras.layers.Dense(1, activation='sigmoid')
	])
# le resultat est pertinant , on a un meilleur model



# Transfert learning

In [ ]:
#tensorflow.org/hub
!pip install --upgrade tensorflow-hub
# utilise 130GB de corpus

In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")
embeddings = embed(["A long sentence.", "single-word", )
embeddings.shape # (2, 20) 2 phrases, 20 dimensions

hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1",
							input_shape=[],
							dtype=tf.string,
							trainable=True)

train_examples = tfds.as_numpy(training_data)

model = tf.keras.models.Sequentials(
	[
		hub_layer,
		tf.keras.layers.Embedding(vocab_size, embedding_dim),
		tf.keras.layers.GlobalAveragePooling1D(),
		tf.keras.layers.Dense(5,activation='relu'),
		tf.keras.layers.Dropout(0.25),
		tf.keras.layers.Dense(3, activation='relu'),
		tf.keras.layers.Dense(1, activation='sigmoid')
	])

# FINAL 

Utiliser Embeddings projector pour se representer
- PCA en 3D pour savoir les mots les plus proches en 3D
